In [1]:
import numpy as np
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def load_data(img_dir, ending):
    return np.array([cv2.imread(os.path.join(img_dir, img)) for img in os.listdir(img_dir) if img.endswith(ending)])


In [ ]:
X_ORG = load_data("3Shapes2_large/", "im1.png")
X_ORG.shape

In [ ]:
X = X_ORG[:10]

In [ ]:
def create_graph():
    pass

In [ ]:
def train():
    pass

In [ ]:
def run():
    pass

In [ ]:
run()